In [136]:
import numpy as np
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df = pd.read_csv("DBData.csv")
df = df.drop(["Unnamed: 20", 'Indicator Code', 'Country Code'], 1)

In [137]:
df = pd.DataFrame(
        df.pivot(index='Indicator Name', 
                 columns='Country Name'
      ).unstack(
      ).reset_index(drop=False))
df.columns = ['Year', 'Country', 'Measure', 'Score']

In [140]:
df.pivot_table(values='Score', index=None, columns=['Year', 'Country', 'Measure'])

AttributeError: 'Series' object has no attribute 'columns'

In [138]:
df

,Year,Country,Measure,Score
0,2004,Afghanistan,Dealing with construction permits (DB06-15 met...,NaN
1,2004,Afghanistan,Dealing with construction permits (DB16-19 met...,NaN
2,2004,Afghanistan,Dealing with construction permits: Building qu...,NaN
3,2004,Afghanistan,Dealing with construction permits: Building qu...,NaN
4,2004,Afghanistan,Dealing with construction permits: Cost (% of ...,NaN
5,2004,Afghanistan,Dealing with construction permits: Cost (% of ...,NaN
6,2004,Afghanistan,Dealing with construction permits: Liability a...,NaN
7,2004,Afghanistan,Dealing with construction permits: Procedures ...,NaN
8,2004,Afghanistan,Dealing with construction permits: Procedures ...,NaN
9,2004,Afghanistan,Dealing with construction permits: Professiona...,NaN


In [99]:
gdp = pd.read_csv('GDPpc.csv')
gdp = gdp.drop(['Country Code', 'Indicator Code'], 1)
gdp = pd.DataFrame(
        gdp.pivot(index='Indicator Name', 
                 columns='Country Name'
      ).unstack(
      ).reset_index(drop=False))
gdp = gdp.drop('Indicator Name', 1)
gdp.columns = ['Year', 'Country', 'GDPperCap']

In [100]:
gdp

,Year,Country,GDPperCap
0,1960,Afghanistan,59.777327
1,1960,Albania,NaN
2,1960,Algeria,244.824806
3,1960,American Samoa,NaN
4,1960,Andorra,NaN
5,1960,Angola,NaN
6,1960,Antigua and Barbuda,NaN
7,1960,Arab World,NaN
8,1960,Argentina,NaN
9,1960,Armenia,NaN


In [90]:
gdp

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.243909e+10,1.585657e+10,1.780429e+10,1.990732e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.904636e+10,1.954398e+10,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192696e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.188368e+10,1.303854e+10,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09,NaN
5,Arab World,ARB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.800000e+12,2.110000e+12,2.500000e+12,2.790000e+12,2.870000e+12,2.910000e+12,2.560000e+12,2.510000e+12,2.590000e+12,NaN
6,United Arab Emirates,ARE,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.540000e+11,2.900000e+11,3.510000e+11,3.750000e+11,3.900000e+11,4.030000e+11,3.580000e+11,3.570000e+11,3.830000e+11,NaN
7,Argentina,ARG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,2.445060e+10,1.827212e+10,2.560525e+10,2.834471e+10,...,3.330000e+11,4.240000e+11,5.300000e+11,5.460000e+11,5.520000e+11,5.260000e+11,5.950000e+11,5.550000e+11,6.370000e+11,NaN
8,Armenia,ARM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.054614e+10,1.153659e+10,NaN
9,American Samoa,ASM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.780000e+08,5.760000e+08,5.740000e+08,6.440000e+08,6.410000e+08,6.430000e+08,6.610000e+08,6.530000e+08,6.340000e+08,NaN
